# Lab

The Benchmark class includes various methods that help in constructing a model, training, evaluating its performance, and visualizing the training progress using TensorFlow. 

Your objective is to implement the MyBenchmark class, which inherits from the Benchmark class and performs simple linear regression.


Follow these steps to complete the task:

* Inherit the MyBenchmark class from the Benchmark class.

* Implement the dataset method to create a simple dataset for linear regression.

* Implement the make_hyper_params method to define hyperparameters for the linear regression model.

* Implement the make_model method to construct the linear regression model using TensorFlow.

* Implement the get_result method to retrieve the training results such as loss.

* Initialize an instance of the MyBenchmark class.

* Transform the input data using the transform_data method.

* Define training parameters, including the number of epochs, batch size, and any required callbacks.

* Train and evaluate the linear regression model using the benchmark method.

* Visualize the training results, such as loss and predictions, using the provided plotting methods.

Additionally, explore and make use of the following advanced features provided by the Benchmark class:

* Save and load the trained model using the save_model and load_model methods.
* Implement model checkpoints to save the model every N epochs or save the best model using the make_checkpoint_callback method.
* Visualize the training progress using TensorBoard by employing the make_tensorboard_callback method.

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


In [18]:


class Benchmark:
    def __init__(self):
        pass

    def dataset(self, train_size=100):
        raise NotImplementedError

    def min_max_scaler(self, data):
        scaler = MinMaxScaler()
        return scaler.fit_transform(data)

    def standard_scaler(self, data):
        scaler = StandardScaler()
        return scaler.fit_transform(data)

    def transform_data(self, x, y, y_label, scaler):
        X = scaler(x)
        Y = scaler(y.reshape(-1, 1))
        return X, Y

    def make_hyper_params(self):
        raise NotImplementedError

    def make_optimizer(self, name, lr):
        print('make_optimizer', name)
        if name == 'Adam':
            return Adam(learning_rate=lr)
        elif name == 'RMSprop':
            return RMSprop(learning_rate=lr)
        elif name == 'SGD':
            return SGD(learning_rate=lr)
        else:
            raise ValueError('Unknown optimizer')

    def make_loss_func(self, name):
        if name == 'MAE':
            return MeanAbsoluteError()
        elif name == 'MSE':
            return MeanSquaredError()
        else:
            raise ValueError('Unknown loss function')

    def make_model(self, opt='Adam', lr='0.01', loss_fn='MSE', **kargs):
        raise NotImplementedError

    def train(self, model, X, y, epochs=50, batch=1, callbacks=None, **kargs):
        model.fit(X, y, epochs=epochs, batch_size=batch, callbacks=callbacks)

    def evaluate(self, model, X_test, y_test):
        return model.evaluate(X_test, y_test)

    def predict(self, model, X):
        return model.predict(X)

    def get_result(self, model, record):
        raise NotImplementedError

    def plot_result(self, data, ylabel, xlabel='epochs'):
        plt.plot(data)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.show()

    def plot_predict(self, model, X_train, y_true):
        y_pred = self.predict(model, X_train)
        plt.plot(y_true, label='True')
        plt.plot(y_pred, label='Predicted')
        plt.legend()
        plt.show()

    def save_model(self, model, filename):
        model.save(filename)

    def load_model(self, filename):
        return tf.keras.models.load_model(filename)

    def make_checkpoint_callback(self, save_best_only=True, period=5):
        filepath = 'model_checkpoint.h5'
        checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, save_best_only=save_best_only,
                                                        save_weights_only=False, monitor='val_loss',
                                                        mode='min', verbose=1, save_freq='epoch', period=period)
        return checkpoint

    def make_tensorboard_callback(self, log_dir='./logs'):
        tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True,
                                                     write_images=True,
                                                     update_freq='epoch', profile_batch=2, embeddings_freq=1)
        return tensorboard

    def benchmark(self, X, y, params=None):
        import sys
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        hyper_params = self.make_hyper_params() if params is None else params
        min_loss = sys.float_info.max
        for param in hyper_params:
            print('*' * 20)
            print('opt: {}, lr: {}, loss_fn: {}, batch: {}' \
                  .format(param['opt'], param['lr'], param['loss_fn'], param['batch']))
            model = self.make_model(**param)
            record = self.train(model, X_train, y_train, **param)
            result = self.get_result(model, record)
            print('result', result)
            score = self.evaluate(model, X_test, y_test)
            if result['loss'][-1] < min_loss:
                print(
                    'loss: {:.2f}, weights: {}, bias: {}'.format(result['loss'][-1], result['weights'], result['bias']))
        return score


In [19]:
class MyBenchmark(Benchmark):
    #  implement your code
    def __init__(self, train_size=100):
        super().__init__()
        self.X, self.y = self.dataset(train_size)

    def dataset(self, train_size=100):
        X = np.linspace(0, 10, train_size)
        y = 3 * X + 5 + np.random.randn(train_size)
        return X.reshape(-1, 1), y.reshape(-1, 1)

    def make_model(self, opt='Adam', lr='0.01', loss_fn='MSE', **kargs):
        model = tf.keras.Sequential(
            [tf.keras.layers.Dense(units=1,
                                   bias_initializer=tf.keras.initializers.Constant(1), \
                                   input_dim=3)])
        #print('opt: {}, lr: {}, loss_fn: {}'.format(opt, lr, loss_fn))
        opt = super().make_optimizer(opt, lr)
        loss = super().make_loss_func(loss_fn)
        model.compile(optimizer=opt, loss=loss)
        return model

    def transform_data(self, x, y, y_label, scaler):
        return super().transform_data(x, y, y_label, scaler)

    def make_checkpoint_callback(self, save_best_only=True, period=5):
        return super().make_checkpoint_callback(save_best_only, period)

    def benchmark(self, X, y, params=None):
        import sys
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        hyper_params = self.make_hyper_params() if params is None else params
        min_loss = sys.float_info.max
        for param in hyper_params:
            print('*' * 20)
            print('opt: {}, lr: {}, loss_fn: {}, batch: {}' \
                  .format(param['opt'], param['lr'], param['loss_fn'], param['batch']))
            model = self.make_model(**param)
            record = self.train(model, X_train, y_train, **param)
            result = self.get_result(model, record)
            print('result', result)
            score = self.evaluate(model, X_test, y_test)
            if result['loss'][-1] < min_loss:
                print(
                    'loss: {:.2f}, weights: {}, bias: {}'.format(result['loss'][-1], result['weights'], result['bias']))
        return score

## Main

In [20]:
my_benchmark = MyBenchmark(train_size=100)
X_transformed, y_transformed = my_benchmark.transform_data(my_benchmark.X, my_benchmark.y, 'Y', my_benchmark.standard_scaler)
train_params = [{
    'epochs': 50,
    'batch': 1,
    'loss_fn': 'MAE',
    'lr': 0.1,
    'opt': 'Adam',
    'callbacks': [
        my_benchmark.make_checkpoint_callback(save_best_only=True, period=5),
        #my_benchmark.make_tensorboard_callback(log_dir='./logs')
    ]
}]
score = my_benchmark.benchmark(X_transformed, y_transformed, train_params)
print(score)

********************
opt: Adam, lr: 0.1, loss_fn: MAE, batch: 1
make_optimizer Adam
Epoch 1/50


ValueError: in user code:

    File "D:\sw15\tensorflow\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "D:\sw15\tensorflow\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\sw15\tensorflow\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "D:\sw15\tensorflow\venv\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "D:\sw15\tensorflow\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\sw15\tensorflow\venv\lib\site-packages\keras\engine\input_spec.py", line 277, in assert_input_compatibility
        raise ValueError(

    ValueError: Exception encou